In [18]:
import numpy as np
import pandas as pd

from PIL import Image

import matplotlib.pyplot as plt


import generate_image_labels as giLabels
import edge_feature_generation as efg

%matplotlib inline
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14,10)

In [43]:
full_df = giLabels.gen_retrain_labels()

In [89]:
def flip( inp_arr ):
    return inp_arr[::-1]

In [110]:
# Go image by image
# pass through image processing pipeline to generate features
# Flip, rotate, each time rot/flipping rc sums and rc ratio
# Dont actually need to rotate or process each image

out_df = pd.DataFrame( 
                        columns=['img_path']+
                                ['rc_ratio']+
                                ['row_'+str(i) for i in range(0,126)]+
                                ['col_'+str(i) for i in range(0,126)]
                     )

img_counter = 0
for i in range( 0, full_df.shape[0] ):

    inp_img = full_df.loc[i,'img_path']
    edge_arr = efg.get_img_edge_data( 
                                inp_img,
                                size=[128,128],
                                edge_cutoff=20,
                                edge_smooth=3,
                                rot_img=False,
                                display=False,
                                ret_arr=True,
                               )

    
    rc_ratio, row, col = efg.get_edge_features( edge_arr, display=False )
    
#    row, col
    row_col_combinations = [
                                [       row  ,       col  ],
                                [       row  , flip( col )],
                                [ flip( row ),       col  ],
                                [ flip( row ), flip( col )],       
                                [       col  ,       row  ],
                                [       col  , flip( row )],
                                [ flip( col ),       row  ], 
                                [ flip( col ), flip( row )],
                           ]

    for j in range( 0, len(row_col_combinations) ):
    
        rc = rc_ratio
        if ( j > 3 ):
            rc = 1./rc_ratio

        out_list = [inp_img,rc,]+row_col_combinations[j][0].tolist()+row_col_combinations[j][1].tolist()
        
        out_df.loc[img_counter,:] = out_list
    
        img_counter = img_counter + 1

In [116]:
foo = pd.merge( full_df, out_df, left_on='img_path', right_on='img_path', how='inner' ) 